In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from os.path import join, exists, expanduser, basename
from genericpath import isdir
from glob import glob
import numpy as np
from PIL import Image
from collections import defaultdict
import matplotlib.pyplot as plt
import torch

# from lib.r2d2.extract import extract_keypoints_modified
from relfm.utils.paths import REPO_PATH
# from relfm.utils.log import print_update, tqdm_iterator
from relfm.utils.visualize import show_images_with_keypoints, set_latex_fonts, show_grid_of_images, get_concat_h
# from relfm.utils.matching import evaluate_matching_with_rotation, analyze_result
# from relfm.inference.r2d2_on_hpatches import configure_save_dir

In [ ]:
set_latex_fonts(show_sample=True)

In [ ]:
DATA_DIR = "../data/hpatches-sequences-release/"

In [ ]:
sequences = glob(join(DATA_DIR, "i_*"))

In [ ]:
for s in sequences:
    # print(basename(s))
    images = glob(join(s, "*.ppm"))
    images = [Image.open(x) for x in images]
    show_grid_of_images(images, n_cols=len(images), figsize=(12, 4), title=basename(s))